# SMDT Pitch: Generate .wav files composed of sine waves (loop)

Please see the similarly named "smdt_pitch_generate_wav-detailed" notebook (should be in the same folder as this one) for step-by-step description of what/why/how stimuli are being generated.

## Imports

In [1]:
import struct
import wave
import numpy as np
import pandas as pd
import os

## Define functions

### gen_amp_movement

Generates a 1-dimensional array of amplitude values.

* amp_max: Maximum amplitude
* amp_move: 'rise'/'descend'/'sustain' - if the amplitude values should describe a movement in volume from min to max ('rise'), max to min ('descend'), or just sustain the same volume (max to max - 'sustain')
* n_samples: Number of values to generate

In [2]:
def gen_amp_movement(amp_max, amp_move, n_samples, amp_min=0):
    if amp_move == 'rise':
        volume_vals = np.linspace(amp_min, amp_max, n_samples)
    elif amp_move == 'descend':
        volume_vals = np.linspace(amp_max, amp_min, n_samples)
    else:
        volume_vals = np.linspace(amp_max, amp_max, n_samples)
    return volume_vals

Generates a sequence of sine wave values, which starts with a rise in volume and ends with a decrease in volume, based on: 
* sampling_rate: sampling rate
* dur_shift: duration of rise/descent
* dur_sustain: duration of sustained volume in middle
* amp_max: maximum amplitude
* freq: wave/sound frequency
* amp_min: minimum amplitude (defaults to 0)

In [3]:
def gen_sine_wave(sampling_rate, dur_shift, dur_sustain, amp_max, freq, amp_min=0):
    tot_n_samples = int(sampling_rate * (dur_shift * 2 + dur_sustain)/1000)
    samples_shift = sampling_rate * (dur_shift / 1000)
    samples_sustain = sampling_rate * (dur_sustain / 1000)
    
    volumes_rise = gen_amp_movement(
        amp_max=amp_max, 
        amp_move='rise', 
        n_samples=samples_shift)
    volumes_sustain = gen_amp_movement(
        amp_max=amp_max, 
        amp_move='sustain', 
        n_samples=samples_sustain)
    volumes_descend = gen_amp_movement(
        amp_max=amp_max, 
        amp_move='descend', 
        n_samples=samples_shift)
    volume_vals = np.concatenate((
        volumes_rise, volumes_sustain, volumes_descend
    ))
    
    sample_nums = np.arange(tot_n_samples)
    seq_vals = volume_vals * np.sin(2 * np.pi * freq * sample_nums / sampling_rate)
    return seq_vals

Generates a sequence of 0 (amplitude) values, representing a quiet pause.

* sampling_rate: sampling rate
* dur_ms: duration (in milliseconds)

In [4]:
def gen_pause_wave(sampling_rate, dur_ms):
    n_samples = int(sampling_rate * dur_ms/1000)
    sample_nums = np.arange(n_samples)
    seq_vals = np.linspace(0, 0, n_samples)
    return seq_vals



## General setup

In [5]:
SAMPLING_RATE = 44100
MAX_AMPLITUDE = 10000
BASE_FREQ = 500
RAMP_DURATION = 30 # milliseconds
SUSTAIN_DURATION = 530 # milliseconds
PAUSE_DURATION = 1000 # milliseconds

save_dir_path = 'generated_wavs/'

## Modify input variables
I'm not entirely sure why we have to do this, it's a dirty fix since I haven't found the root problem. It's related to when we repeat values after the loop for producing stereo sound. That seems to also lead to doubled play time, and so we have to cut the durations in half here.

In [6]:
# RAMP_DURATION /= 2
# SUSTAIN_DURATION /= 2
# PAUSE_DURATION /= 2

## Loop

In [7]:
for odd_freq_index in range(2):
    sound_freqs = [BASE_FREQ, BASE_FREQ]
    for add_hz in range(1, 18):
        sound_freqs[odd_freq_index] = BASE_FREQ + add_hz
        
        # first sound
        seq1 = gen_sine_wave(
            sampling_rate=SAMPLING_RATE, 
            dur_shift=RAMP_DURATION,
            dur_sustain=SUSTAIN_DURATION,
            amp_max=MAX_AMPLITUDE,
            freq=sound_freqs[0]
        )
        
        # pause
        seq_pause = gen_pause_wave(
            sampling_rate=SAMPLING_RATE, 
            dur_ms=PAUSE_DURATION
            )
        
        # second sound
        seq2 = gen_sine_wave(
            sampling_rate=SAMPLING_RATE, 
            dur_shift=RAMP_DURATION,
            dur_sustain=SUSTAIN_DURATION,
            amp_max=MAX_AMPLITUDE,
            freq=sound_freqs[1]
        )
        
        # merge all sequences into one
        end_seq = np.concatenate((
            seq1, seq_pause, seq2
        ), axis=0
        )
        
        # repeat each value once (since we're using stereo channels)
        end_seq = np.repeat(end_seq, 2)
        
        # export to WAV
        save_file_path = os.path.join(save_dir_path, f'smdtpitch_{sound_freqs[0]}_{sound_freqs[1]}.wav')
        noise_output = wave.open(save_file_path, 'w')
        noise_output.setparams((2, 2, 44100, 0, 'NONE', 'not compressed'))

        values = [struct.pack('h', int(value)) for value in end_seq]

        value_flat = b''.join(values)
        noise_output.writeframes(value_flat)
        noise_output.close()